# FINM 35000 Problem Set 3: Equity Valuation Stress Testing

<span style="color:blue">Aman Krishna </span> <br>
<br>
<span style="color:#406A5F">Tim Taylor </span> <br>
<br>
<span style="color:purple">Yazmin Ramirez Delgado </span>

In [14]:
import pandas as pd
import numpy as np


## Stress testing an equity portfolio (100 points)

Project market value changes of an equity portfolio from time0 to time1 based on the changes in the macroeconomic variables (MEVs) from the severely adverse economic domestic scenario of the Federal Reserve’s Comprehensive Capital Analysis and Review (CCAR) 2021. Time0 is 12/31/2020 and time1 is 12/31/2021.

To make the projections, you may need to map the changes of MEVs from time0 to time1 to the change in values of the stocks from time0 to time1. In addition to the scenario given above,

    - Investment instruments: a portfolio of 20 stocks, 10 million shares in each stock.
    - Additional scenarios and historical data of the MEVs in the Fed scenarios. The Federal Reserve also provide scenario description narratives.
    - Equity and Fama-French factor historical data (total returns), ‘returns’ tab in wrds_data.xlsx

## 1. Build three projection models then compare the approaches and project the changes in the portfolio’s value: CAPM, Fama-French, and a “general multi-factor” model. You are required to provide support for your modeling choices where applicable (segmentation3, variable selection, choice(s) of historical data window, etc.). Your report should provide a clear picture of the modeling development process and modeling choices:

In [15]:
# Load the data
wrds_data = pd.read_excel('wrds_data-1.xlsx', sheet_name='returns')
#Change Date column fromth format "YYYY-MM" to datetime
wrds_data['Date'] = pd.to_datetime(wrds_data['Date'], format='%Y-%m')
#remove the columns after "WALMART INC"
wrds_data = wrds_data.iloc[:,0:28]

#Calculate excess return for each stock starting column 8. The risk free rate is in column 4
wrds_data.iloc[:,8:] = wrds_data.iloc[:,8:].sub(wrds_data.iloc[:,4], axis=0)

#Resample the data to quarterly with the first day of the quarter as the date
wrds_data_q = wrds_data.set_index('Date').resample('QS').first().reset_index()
wrds_data_q.head()

,Date,MKT COMPOSITE RETURN,S&P RETURN,FAMA-FRENCH MARKET FACTOR,RISK-FREE RATE,FAMA-FRENCH SIZE FACTOR (SMB),FAMA-FRENCH VALUE FACTOR (HML),MOMENTUM FACTOR,BIOGEN INC,JOHNSON & JOHNSON,...,ARCHER-DANIELS-MIDLAND CO,CONAGRA BRANDS INC,COLGATE-PALMOLIVE CO,CAMPBELL SOUP CO,KELLOGG CO,KRAFT HEINZ CO,COCA-COLA CO,PROCTER & GAMBLE CO,TYSON FOODS INC -CL A,WALMART INC
0,1963-01-01,NaN,0.049,0.049,0.003,0.031,0.022,-0.021,NaN,0.100,...,NaN,NaN,0.070,0.091,0.149,NaN,0.056,0.031,NaN,NaN
1,1963-04-01,NaN,0.049,0.045,0.003,-0.013,0.010,-0.001,NaN,0.009,...,NaN,NaN,0.043,0.001,-0.055,NaN,-0.013,0.054,NaN,NaN
2,1963-07-01,NaN,-0.004,-0.004,0.003,-0.005,-0.009,0.010,NaN,-0.005,...,NaN,NaN,-0.003,0.036,0.062,NaN,0.016,-0.030,NaN,NaN
3,1963-10-01,NaN,0.032,0.025,0.003,-0.006,-0.000,0.031,NaN,0.157,...,NaN,NaN,-0.088,0.013,0.070,NaN,0.012,0.005,NaN,NaN
4,1964-01-01,NaN,0.027,0.022,0.003,-0.002,0.015,0.011,NaN,0.029,...,NaN,NaN,-0.038,-0.028,-0.026,NaN,0.019,0.031,NaN,NaN


In [16]:
#Backfill the missing values
wrds_data = wrds_data.fillna(method='bfill')
wrds_data_q = wrds_data_q.fillna(method='bfill')
wrds_data.head()

,Date,MKT COMPOSITE RETURN,S&P RETURN,FAMA-FRENCH MARKET FACTOR,RISK-FREE RATE,FAMA-FRENCH SIZE FACTOR (SMB),FAMA-FRENCH VALUE FACTOR (HML),MOMENTUM FACTOR,BIOGEN INC,JOHNSON & JOHNSON,...,ARCHER-DANIELS-MIDLAND CO,CONAGRA BRANDS INC,COLGATE-PALMOLIVE CO,CAMPBELL SOUP CO,KELLOGG CO,KRAFT HEINZ CO,COCA-COLA CO,PROCTER & GAMBLE CO,TYSON FOODS INC -CL A,WALMART INC
0,1963-01-01,-0.009,0.049,0.049,0.003,0.031,0.022,-0.021,-0.017,0.100,...,0.001,0.112,0.070,0.091,0.149,-0.086,0.056,0.031,-0.035,0.091
1,1963-02-01,-0.009,-0.029,-0.024,0.002,0.005,0.022,0.025,-0.017,0.017,...,0.001,0.112,-0.002,-0.106,-0.062,-0.086,-0.005,-0.033,-0.035,0.091
2,1963-03-01,-0.009,0.035,0.031,0.002,-0.026,0.021,0.016,-0.017,0.001,...,0.001,0.112,0.070,0.010,0.097,-0.086,0.044,0.040,-0.035,0.091
3,1963-04-01,-0.009,0.049,0.045,0.003,-0.013,0.010,-0.001,-0.017,0.009,...,0.001,0.112,0.043,0.001,-0.055,-0.086,-0.013,0.054,-0.035,0.091
4,1963-05-01,-0.009,0.014,0.018,0.002,0.011,0.025,0.004,-0.017,0.049,...,0.001,0.112,0.008,0.019,0.096,-0.086,0.003,-0.018,-0.035,0.091


In [17]:
def convert_to_datetime(quarterly_date):
    year, quarter = quarterly_date.split(' Q')
    year = int(year)
    quarter = int(quarter)
    
    # Calculate the month by mapping quarter to months (e.g., Q1 -> January)
    month = 3 * quarter - 2
    
    # Create a datetime object with the year and month
    return pd.to_datetime(f'{year}-{month:02d}')


In [18]:
# Loading the MEVs
df1 = pd.read_csv('2021-table_1b_historic_international.csv')
df2 = pd.read_csv('2021-table_1a_historic_domestic.csv')

# Drop the Scenario Name column
df1 = df1.drop(columns=['Scenario Name'])
df2 = df2.drop(columns=['Scenario Name'])

df1['Date'] = df1['Date'].apply(convert_to_datetime)
df2['Date'] = df2['Date'].apply(convert_to_datetime)

In [19]:
#Check shape before merging
df1.shape, df2.shape

((180, 13), (180, 17))

In [20]:
#Check shape of wrds_data and df_mev
wrds_data.shape, wrds_data_q.shape

((696, 28), (232, 28))